## Get bandwidth from cv part files

In [1]:
#%matplotlib notebook

import cPickle as pickle
import glob
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os.path
from matplotlib import ticker

from kde_classes import Model, KDE

Welcome to JupyROOT 6.14/04


In [3]:
#gammas = np.linspace(2.0, 3.0, 11)
gammas = np.concatenate(([1.5], np.linspace(2.0, 3.0, 11)))
#gammas = [3.0]
module = 'sig_psi_E'
model = Model(module, mc=None, weighting='plotter_wkde')
kde = KDE(model)

for gamma in gammas:
    cv_files = glob.glob('output/{module}/adaptive_kd_plotter_wkde_gamma_{gamma}_phi0_1.0/cv/cv_*.npy'.format(module=module, gamma=gamma))
    #cv_files = glob.glob('output/sig_psi_E/old/adaptive/cv/cv_*.npy')

    cv_results_split = np.array([], dtype=kde.cv_result_dtype)
    #cv_results_split = np.array([], dtype=np.load(cv_files[0]).dtype)

    for cv_file in cv_files:
        cv_result_split = np.load(cv_file)
        #print(cv_result_split)
        cv_results_split = np.append(cv_results_split, cv_result_split)

    # Gather splitted cv results by calculating average values.
    arr, unique_index = np.unique(cv_results_split['bandwidth'], return_index=True,
                                  axis=0)
    cv_results = cv_results_split[unique_index]
    for i, cv_result in enumerate(cv_results):
        matches = cv_results_split[np.all(
            cv_results_split['bandwidth'] == cv_result['bandwidth'], axis=1)]
        cv_results['LLH'][i] = np.average(matches['LLH'])
        cv_results['Zeros'][i] = np.average(matches['Zeros'])
    # cv_results = cv_results_split
    cv_results_max_LLH = cv_results[cv_results['LLH'] == np.max(cv_results['LLH'])]
    print("Gamma: {}; CV: {}".format(gamma, cv_results_max_LLH))
    bandwidth = cv_results_max_LLH['bandwidth'][0]

#     plt.figure(dpi=150)
#     plt.tricontourf(cv_results['bandwidth'][:, 0], cv_results['bandwidth'][:, 1],
#                     cv_results['LLH'], 50)
#     plt.colorbar()
#     plt.scatter(bandwidth[0], bandwidth[1], marker='x', label=bandwidth)
#     plt.title('$\gamma$={}'.format(gamma))
#     plt.xlabel('bw of sigma_pull_corrected')
#     plt.ylabel('bw of psi')
#     plt.legend()
    
#     plt.figure(dpi=150)
#     plt.tricontourf(cv_results['bandwidth'][:, 1], cv_results['bandwidth'][:, 2],
#                     cv_results['LLH'], 50)
#     plt.colorbar()
#     plt.scatter(bandwidth[1], bandwidth[2], marker='x', label=bandwidth)
#     plt.title('$\gamma$={}'.format(gamma))
#     plt.xlabel('bw of psi')
#     plt.ylabel('bw of log_e')
#     plt.legend()
    
#     plt.figure(dpi=150)
#     plt.tricontourf(cv_results['bandwidth'][:, 0], cv_results['bandwidth'][:, 2],
#                     cv_results['LLH'], 50)
#     plt.colorbar()
#     plt.scatter(bandwidth[0], bandwidth[2], marker='x', label=bandwidth)
#     plt.title('$\gamma$={}'.format(gamma))
#     plt.xlabel('bw of sigma_pull_corrected')
#     plt.ylabel('bw of log_e')
#     plt.legend()

#     plt.figure(dpi=150)
#     plt.plot(cv_results['bandwidth'][:, 1], cv_results['LLH'])
#     plt.title('$\gamma$={}'.format(gamma))
#     plt.xlabel('bw of psi')
#     plt.ylabel('LLH')

Gamma: 1.5; CV: [([0.17, 0.42, 0.6 ], -1.9458729, 0.)]
Gamma: 2.0; CV: [([0.13, 0.34, 0.26], -1.7132763, 0.)]
Gamma: 2.1; CV: [([0.11, 0.32, 0.3 ], -1.6474864, 0.)]
Gamma: 2.2; CV: [([0.16, 0.24, 0.31], -1.588302, 0.)]
Gamma: 2.3; CV: [([0.12, 0.28, 0.3 ], -1.5319114, 0.)]
Gamma: 2.4; CV: [([0.13, 0.28, 0.21], -1.4740968, 0.)]
Gamma: 2.5; CV: [([0.14, 0.26, 0.17], -1.4158982, 0.)]
Gamma: 2.6; CV: [([0.14, 0.26, 0.17], -1.3592956, 0.)]
Gamma: 2.7; CV: [([0.12, 0.26, 0.15], -1.3043567, 0.)]
Gamma: 2.8; CV: [([0.15, 0.26, 0.13], -1.252952, 0.)]
Gamma: 2.9; CV: [([0.15, 0.22, 0.13], -1.2037599, 0.)]
Gamma: 3.0; CV: [([0.13, 0.24, 0.07], -1.1574348, 0.)]


In [ ]:
#np.set_printoptions(threshold=np.inf)
print(cv_results['LLH'][cv_results['bandwidth'][:, 2] == 0.2])
# print(cv_results_max_LLH)
# bandwidth = cv_results_max_LLH['bandwidth']

In [ ]:
for bw in np.unique(cv_results['bandwidth'][:, 2]):
    print(bw)
    print(cv_results[cv_results['bandwidth'][:, 2] == bw])
    print(np.unique(cv_results['bandwidth'][:, 0][cv_results['bandwidth'][:, 2] == bw]))
    if len(np.unique(cv_results['bandwidth'][:, 0][cv_results['bandwidth'][:, 2] == bw])) != 1 and\
            len(np.unique(cv_results['bandwidth'][:, 1][cv_results['bandwidth'][:, 2] == bw])) != 1:
        plt.figure(dpi=150)
        plt.tricontourf(cv_results['bandwidth'][cv_results['bandwidth'][:, 2] == bw][:, 0],
                        cv_results['bandwidth'][cv_results['bandwidth'][:, 2] == bw][:, 1],
                        cv_results['LLH'][cv_results['bandwidth'][:, 2] == bw],
                        50)
        plt.colorbar()
        # plt.scatter(bandwidth[1], bandwidth[2], marker='x', label=bandwidth)
        plt.title('$\gamma$={}, bw of log_e={:.2f}'.format(gamma, bw))
        plt.xlabel('bw of sigma_pull_corrected')
        plt.ylabel('bw of psi')
        #plt.legend()


In [ ]:
plt.figure(dpi=150)
plt.plot(cv_results['bandwidth'][:, 1], cv_results['LLH'])
plt.title('$\gamma$={}'.format(gamma))
plt.xlabel('bw of psi')
plt.ylabel('LLH')

## Plot contours of CV

In [ ]:
plt.figure(dpi=150)
plt.tricontourf(cv_results['bandwidth'][:, 0], cv_results['bandwidth'][:, 1],
                cv_results['LLH'], 50)
plt.colorbar()
plt.scatter(bandwidth[0], bandwidth[1], marker='x', label=bandwidth)
plt.legend()
plt.title('$\gamma$={}'.format(gamma))
plt.xlabel('bw of log(sigma_pull_corrected)')
plt.ylabel('bw of log_e')

In [ ]:
from scipy.interpolate import griddata

x = cv_results['bandwidth'][:, 0]
y = cv_results['bandwidth'][:, 1]
z =  cv_results['LLH']

xi = np.unique(x)
yi = np.unique(y)
zi = griddata((x, y), z, (xi[None,:], yi[:,None]), method='linear')

plt.figure(dpi=150)
plt.contourf(xi, yi, zi, 20)
plt.colorbar()
plt.scatter(bandwidth[0], bandwidth[1], marker='x', label=bandwidth)
plt.legend()

## dataset_8yr_fit_IC86_2012_16_MC_2017_09_29_more_fields names

In [ ]:
mc = np.load('/home/ge56lag/Data/dataset_8yr_fit_IC86_2012_16_MC_2017_09_29_more_fields.npy')
mc.dtype.names

In [ ]:
np.linspace(0.01, 0.25, 5)

In [ ]:
np.geomspace(0.01, 0.25, 5)

In [ ]:
import cPickle as pickle
import glob
import itertools
import numpy as np
import os.path

from kde_classes import Model, KDE

model = Model('dec', mc=None, weighting=None)
kde = KDE(model)

cv_files = glob.glob('output/dec/temp/cv_*.npy')
cv_results_split = np.array([], dtype=kde.cv_result_dtype)
#cv_results = np.array([], dtype=kde.cv_result_dtype)

for cv_file in cv_files:
    cv_result_split = np.load(cv_file)
    cv_results_split = np.append(cv_results_split, cv_result_split)

# Gather splitted cv results by calculating average values.
arr, unique_index = np.unique(cv_results_split['bandwidth'], return_index=True,
                              axis=0)
cv_results = cv_results_split[unique_index]
for i, cv_result in enumerate(cv_results):
    matches = cv_results_split[np.all(
        cv_results_split['bandwidth'] == cv_result['bandwidth'], axis=1)]
    cv_results['LLH'][i] = np.average(matches['LLH'])
    cv_results['Zeros'][i] = np.average(matches['Zeros'])

cv_results_max_LLH = cv_results[cv_results['LLH'] == np.max(cv_results['LLH'])]

print(cv_results_max_LLH)
#bandwidth = [cv_results_max_LLH[key] for key in model.bandwidth_vars]


In [ ]:
cv_results_split

In [ ]:
cv_results